<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_causalLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#因果模型每个句子都需要添加一个eos_token，基于上文token预测当前token

In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/nlp")

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
from datasets import Dataset,load_dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForLanguageModeling,Trainer,TrainingArguments,pipeline

In [8]:
datasets = load_dataset("pleisto/wikipedia-cn-20230720-filtered",cache_dir="./")
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['completion', 'source'],
        num_rows: 254547
    })
})

In [11]:
datasets["train"][0]["completion"]

'昭通机场（ZPZT）是位于中国云南昭通的民用机场，始建于1935年，1960年3月开通往返航班“昆明－昭通”，原来属军民合用机场。1986年机场停止使用。1991年11月扩建，于1994年2月恢复通航。是西南地区「文明机场」，通航城市昆明。 机场占地1957亩，飞行区等级为4C，有一条跑道，长2720米，宽48米，可供波音737及以下机型起降。机坪面积6600平方米，停机位2个，航站楼面积1900平方米。位于城东6公里处，民航路与金鹰大道交叉处。\n航点\n客服电话\n昭通机场客服电话：0870-2830004'

In [10]:
def data_process(examples):
  context = [e + tokenizer.eos_token for e in examples["completion"]]
  return tokenizer(context,max_length=64,truncation=True)
ds = datasets["train"].map(data_process,remove_columns=datasets["train"].column_names,batched=True)

Map:   0%|          | 0/254547 [00:00<?, ? examples/s]

In [12]:
ds[0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
ds[0]["input_ids"]

In [14]:
tokenizer.eos_token_id

2

In [18]:
args=TrainingArguments(
    output_dir="./causal_lm",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=50
)
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

Step,Training Loss
10,2.737900
20,2.934500
30,2.710100
40,2.379600
50,2.393500


KeyboardInterrupt: 

In [19]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/nlp/causal_lm/checkpoint-50")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
pipe("我是一名受人尊敬的德高望重的医生", max_length=128, do_sample=True)

[{'generated_text': '我是一名受人尊敬的德高望重的医生，我爱我的工作，我爱我的家庭，我爱爱我的人生。每当我感到困惑或悲伤时，我都会祈祷你平安。\n我经常遇到困惑或悲伤时，我都会祈祷。它们可能是因为工作的复杂性，工作的残酷性，工作的不稳定性，工作的复杂性，或者其他各种原因，有时是因为我的家人或我的社会。我的家人可能会因为我的工作而感到困惑或悲伤，有时候我的家人或我的公司也可能会遇到类似的情况。我的家人或公司通常都向我推荐德高望重的医生，或者由我推荐的其他人，来'}]